# MODEL TESTING

The purpose of this notebook is to test different model learning configurations. Runs are saved in the standard _runs_ directory and can be referenced by their unique ID (directory name). Each run-directory contains the following files from training:
- manager.pkl: run manager object containing all settings of the model run - see implementation _/helpers/run_manager.py_ for details
- checkpoint_last.pth: dict containing the most recent state dict for both the model and optimizer (keys: epoch, model, optimizer)
- checkpoint_best.pth: same dict as above, but the best performing model over the entire run based on loss
- ... and randomly named files from the tensorboard summary writer

In [1]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import os
import shutil
import numpy as np
import pandas as pd

import models
import datasets
from helpers import io, run_manager, tester

In [2]:
EXP_ID = "basic"

DATA_PATH = "/home/jovyan/work/DENMARK/256x256"
SAVE_PATH = "/home/jovyan/work/runs"

EM = io.loadPKL(os.path.join(SAVE_PATH, EXP_ID, "manager.pkl"))
TB = SummaryWriter(EM.save_path)

## Load Model from Run Config

In [ ]:
model = models.getNet(EM.net_name, EM.n_classes).cuda()

checkpoint = torch.load(os.path.join(EM.save_path, 'checkpoint_best.pth'))
model.load_state_dict(checkpoint['model'])

print("Model loaded...")

## Load Testing Data

In [3]:
# load image-lists from files
images_path = os.path.join(DATA_PATH, 'image_sets', 'shapes.txt')
images_list = [name.replace("\n","") for name in io.readText(images_path)]
test_size = len(images_list) #round(len(images_list) * 0.7)
test_images = images_list[:test_size]

print(f"Dataset sizes: \n - test: {len(test_images)}")

Dataset sizes: 
 - test: 378


In [4]:
test_set = datasets.getDataset(name = 'denmark_shapes',
                               path = DATA_PATH,
                               images = test_images,
                               n_classes = EM.n_classes,
                               transform = EM.transform)

test_sampler = torch.utils.data.SequentialSampler(test_set)

test_loader = DataLoader(test_set, sampler = test_sampler,
                         batch_size = 3,
                         num_workers = 2)

print("Dataloaders ready...")

Dataloaders ready...


## Run Tests

In [ ]:
metric_list = ["mIoU", "dice", "pAccuracy"] #mIoU, dice, pAccuracy

results_dict = tester.testModel(model, test_loader, metric_list)
tester.imagesToTB(model, test_loader, TB, 30)
EM.addTest(results_dict)
io.savePKL(os.path.join(EM.save_path, 'manager.pkl'), EM)

print(pd.DataFrame(results_dict, index=[0]))

# EXPLORATION REGION

In [ ]:
EM.test_dicts = None

In [5]:
dataiter = iter(test_loader)
batch = dataiter.next()

In [9]:
batch['meta']['path'][0]

'/home/jovyan/work/DENMARK/256x256/images/2019_1km_6078_637_2.jpg'

In [ ]:
from helpers import haven_viz
img_src = haven_viz.get_image(images.cpu(), denorm="rgb")
y_list, x_list = np.where(points_numpy.squeeze())

imagesl = []
for blob in point_blobs:
    if blob == 0:
        continue
    blob_mask = blobs==blob
    pred_img = haven_viz.mask_on_image(img_src, blob_mask.squeeze()))
    

img_labels = haven_viz.points_on_image(y_list, x_list, img_src)
img_pred = haven_viz.mask_on_image(img_src, blobs.squeeze())
img_pred_shape = haven_viz.mask_on_image(img_src, np.array(img).squeeze())

stitched_image = np.hstack([img_labels, img_pred, img_pred_shape])
stitched_image2 = np.hstack(imagesl)
haven_viz.save_image('test.jpg', stitched_image)
haven_viz.save_image('test2.jpg', stitched_image2)

In [ ]:
# TO WRITE IMAGE TO TENSORBOARD

# get some random training images
dataiter = iter(train_loader)
images = dataiter.next()['images']

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# write to tensorboard
TB.add_image('test', img_grid)